# Problem with acceleration when thrust

## Purpose
* Explore why the thrust is making the acceleration not work

## Methodology
* Simulate a straight course with thrust

## Results
Describe and comment the most important results.

## Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_simple_nonlinear  as vmm
## Examples
from vessel_manoeuvring_models.data.case_1 import ship_parameters, df_parameters, ps, ship_parameters_prime

## Ship parameters

In [ ]:
ship_parameters

## Brix parameters

In [ ]:
mask = df_parameters['prime'].notnull()
index = df_parameters.loc[mask,'prime'].index
coefficients=vmm.simulator.get_all_coefficients(sympy_symbols=False)
missing_coefficients = set(coefficients) - set(index)
missing_coefficients

In [ ]:
mask = df_parameters['prime'].notnull()
df_parameters.loc[mask,'prime']

## Simulate data

In [ ]:
parameters=df_parameters['prime'].copy()
parameters.Xuu=0

t_ = np.linspace(0,10,1000)
df_ = pd.DataFrame(index=t_)

df_['u'] = 2
df_['v'] = 0
df_['r'] = 0
df_['x0'] = 0
df_['y0'] = 0
df_['psi'] = 0
df_['U'] = np.sqrt(df_['u']**2 + df_['v']**2)
df_['beta'] = -np.arctan2(df_['v'],df_['u'])
df_['thrust'] = 30

df_['delta'] = 0

result = vmm.simulator.simulate(df_=df_, parameters=parameters, ship_parameters=ship_parameters, 
                                  control_keys=['delta','thrust'], primed_parameters=True,prime_system=ps)

df_result = result.result.copy()

In [ ]:
ps.prime(df_, U=df_['U']).head()

In [ ]:
result.track_plot(compare=False);

In [ ]:
result.plot(compare=False);

### Check accelerations

In [ ]:
import scipy.integrate

In [ ]:
u_ = df_result['u'].values
u1d_ = df_result['u1d'].values

t_ = np.array(df_result.index)

u_integrated = u_[0] + scipy.integrate.cumtrapz(y=u1d_, x=t_)
u_integrated = np.concatenate([[u_[0]], u_integrated])

fig,ax=plt.subplots()
df_result.plot(y='u', ax=ax)
ax.plot(t_, u_integrated, '--', label='u_integrated')
ax.legend()